### call-your-own-api-inside-ai-assistant-openai-function-calling

In [1]:
import dotenv
%load_ext dotenv
%dotenv

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from openai import OpenAI
client = OpenAI()

In [ ]:
# [81.206732,26.706858,81.211581,26.710769]

In [54]:
get_crop_yeild = {
    "name" : "get_crop_yeild",
    "description" : "Function takes in Date and Location and give crop yeild prediction",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "Date": {
                "type":"string",
                "description" : "Datetime for which data needs to be fetched"
            },
            "Location":{
                "type" : "string",
                "description" : "Location for which data needs to be fetched"
            },
            "BBOX":{
                "type" : "array",
                "items": {
                            "type": "number"
                            },
                "description" : "Bounding Box for the Location for which data needs to be fetched"
            }
        },
        "required" : ["Date","BBOX"]
    }
}

In [55]:
assistant = client.beta.assistants.create(
    instructions="You are a bot who works like a code-interpreter and task executor",
    model="gpt-3.5-turbo-1106",
    tools = [{"type":"function", "function":get_crop_yeild}]
)
assistant

Assistant(id='asst_5XbfVUNGhxina0m6xUOqnrcI', created_at=1718101830, description=None, instructions='You are a bot who works like a code-interpreter and task executor', metadata={}, model='gpt-3.5-turbo-1106', name=None, object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='get_crop_yeild', description='Function takes in Date and Location and give crop yeild prediction', parameters={'type': 'object', 'properties': {'Date': {'type': 'string', 'description': 'Datetime for which data needs to be fetched'}, 'Location': {'type': 'string', 'description': 'Location for which data needs to be fetched'}, 'BBOX': {'type': 'array', 'items': {'type': 'number'}, 'description': 'Bounding Box for the Location for which data needs to be fetched'}}, 'required': ['Date', 'BBOX']}), type='function')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)

In [56]:
thread = client.beta.threads.create()
thread

Thread(id='thread_xDUnDFfTqRWgLk71Av8i0P7c', created_at=1718101833, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [57]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content='What is the crop yield for area with [81.206732,26.706858,81.211581,26.710769] on 2023-11-17?'
)

In [58]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [59]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id,
)

In [60]:
tool_calls = run.required_action.submit_tool_outputs.tool_calls
print(tool_calls)

[RequiredActionFunctionToolCall(id='call_SvuwhQRmJmLL9rdJHCx6NF2A', function=Function(arguments='{"Date":"2023-11-17","BBOX":[81.206732,26.706858,81.211581,26.710769]}', name='get_crop_yeild'), type='function')]


In [120]:
import json
import datetime
import numpy as np
from sentinelhub import SHConfig, CRS, BBox, DataCollection, MimeType, WcsRequest, CustomUrlParam
import rasterio
import requests
from rasterstats import zonal_stats
import geopandas as gpd

# Extract arguments from the input
args = json.loads(tool_calls[0].function.arguments)
Date, BBOX = date, bbox

if tool_calls[0].function.name == "get_crop_yeild":
    def cropYeild(date: str, bbox: list) -> str:
        """Calculate crop yield for date and bbox provided"""

        INSTANCE_ID = '924b0ee0-f5a0-4b73-b2fb-f4edb8ab196d'  # Replace with your actual instance ID

        if INSTANCE_ID:
            config = SHConfig()
            config.instance_id = INSTANCE_ID
        else:
            config = None

        formatted_time = str(date)

        def coordinates_calc():
            return bbox

        def dimensions():
            extent = coordinates_calc()
            bbox = BBox(bbox=extent, crs=CRS.WGS84)
            FAPAR = fapar_layer_call(bbox)
            transform = rasterio.transform.from_bounds(extent[0], extent[1], extent[2], extent[3], FAPAR.shape[1], FAPAR.shape[0])
            return bbox, transform

        def fapar_layer_call(bbox):
            wcs_request = WcsRequest(
                data_collection=DataCollection.SENTINEL2_L2A,
                layer='FAPAR',
                bbox=bbox,
                time=formatted_time,
                resx='10m',
                resy='10m',
                image_format=MimeType.TIFF,
                custom_url_params={CustomUrlParam.SHOWLOGO: False},
                config=config
            )
            return wcs_request.get_data()[0]

        def LSWI_layer_call(bbox):
            wcs_request = WcsRequest(
                data_collection=DataCollection.SENTINEL2_L2A,
                layer='LSWI',
                bbox=bbox,
                time=formatted_time,
                resx='10m',
                resy='10m',
                image_format=MimeType.TIFF,
                custom_url_params={CustomUrlParam.SHOWLOGO: False},
                config=config
            )
            return wcs_request.get_data()[0]

        def w_calc():
            bbox, transform = dimensions()
            LSWI = LSWI_layer_call(bbox)
            LSWI_max = np.amax(LSWI)
            w = (1 - LSWI) / (1 + LSWI_max)
            return w, transform

        def lue_calc():
            e0 = 3.22
            w, transform = w_calc()
            LUE = e0 * w
            return LUE, transform

        def ghi_calc():
            bbox, transform = dimensions()
            centroid_x = (bbox.min_x + bbox.max_x) / 2
            centroid_y = (bbox.min_y + bbox.max_y) / 2
            API_KEY = 'your-openweathermap-api-key'  # Replace with your actual OpenWeatherMap API key
            response = requests.get(f"https://api.openweathermap.org/energy/1.0/solar/data?lat={centroid_y}&lon={centroid_x}&date={formatted_time}&appid={API_KEY}")
            data = response.json()
            GHI = 0
            for i in data["irradiance"]["daily"]:
                GHI = i['clear_sky']['ghi']
            GHI = GHI / 1000 * 3.6
            return GHI

        def npp_calc():
            bbox, transform = dimensions()
            FAPAR = fapar_layer_call(bbox)
            GHI = ghi_calc()
            LUE, transform = lue_calc()
            NPP = FAPAR * GHI * 0.5 * LUE
            NPP[NPP < 0] = 0
            return NPP

        def cy_calc():
            NPP = npp_calc()
            cy = NPP * 0.8 * 10
            return cy

        def save_tiff(data, transform, path):
            with rasterio.open(path, 'w', driver='GTiff', width=data.shape[1], height=data.shape[0], count=1, dtype=data.dtype, crs='EPSG:4326', transform=transform) as dst:
                dst.write(data, 1)

        cy = cy_calc()
        save_tiff(cy, dimensions()[1], 'data/cy.tiff')

        shp = gpd.read_file("data/output.shp").to_crs(epsg=4326)
        with rasterio.open("data/cy.tiff") as src:
            out_image, out_transform = rasterio.mask.mask(src, shp.geometry, crop=True)
            out_meta = src.meta.copy()
            out_meta.update({"driver": "GTiff", "height": out_image.shape[1], "width": out_image.shape[2], "transform": out_transform})

        with rasterio.open("data/cy.tiff", "w", **out_meta) as dest:
            dest.write(out_image)

        def zonal_stats_calc():
            stats = zonal_stats("data/output.shp", "data/cy.tiff", stats="mean")
            return stats[0]['mean']

        cy_mean = zonal_stats_calc()
        return round(cy_mean, 2)

    response = cropYeild(date, bbox)

    tool_outputs = []
    tool_outputs.append({"tool_call_id": tool_calls[0]['id'], "output": response})
    print(tool_outputs)


IndexError: list index out of range

In [ ]:
import json
args = json.loads(tool_calls[0].function.arguments)
Date, BBOX = date, bbox
if tool_calls[0].function.name == "get_crop_yeild":
    # place to make a call and fetch data
    import datetime
    def cropYeild(date: str, y: list) -> str:
        """Calculate crop yeild for date and bbox provided"""
        """ Steps of Action ####

        ## 1. Trigger Data download from sentinel Hub OGC API using the input shapefile
        ## 2. accordingly download FAPAR, LSWI,  from sentinel Hub
        ## 3. Calculate W from LSWI
        ## 4. Use 24 hour sum og GHI and then it'll be Wh/m2 and then convert in kwh/m2
        ## 5. Using these LUE values as meyion and , calculate NPP
        ## 6. Mention HArvest Index (0.8) and caluclate final yield
        ## 7. Do a multiplication factor (10) to convert tons per hectare
        ## 8. Save pixel wise 
        ## 9. Perform zonal statistics and calculate avg yield and this will be final field wise yield (ton/hectare) """
        
        print("cropyieldscript")

        import sentinelhub
        from sentinelhub import SHConfig


        import datetime
        import numpy as np
        from sentinelhub import CRS, BBox, DataCollection, MimeType, WcsRequest, WmsRequest
        import pyproj
        from IPython import get_ipython
        from sentinelhub import CustomUrlParam
        import warnings
        import geopandas as gpd
        import rasterio
        import rasterio.mask
        from sentinelhub import CRS, BBox, DataCollection, MimeType, WcsRequest, WmsRequest, WebFeatureService, CustomUrlParam
        from rasterio import plot
        from PIL import Image
        import requests
        import json
        import pyproj
        import geopandas as gpd
        from rasterstats import zonal_stats
        from datetime import datetime, date
        warnings.filterwarnings("ignore")
       
        formatted_time = date
        print(formatted_time)
        INSTANCE_ID = '924b0ee0-f5a0-4b73-b2fb-f4edb8ab196d'  # In case you put instance ID into configuration file you can leave this unchanged

        if INSTANCE_ID:
            config = SHConfig()
            config.instance_id = INSTANCE_ID
        else:
            config = None

        file_path = r'geojson_data.json'

        gdf = gpd.read_file(file_path)
        print(gdf)
        gdf.to_file('data/output.shp')
        tmp = gpd.read_file('data/output.shp')

        # print(tmp.head())

        with open(file_path, 'r') as f:
                geojson_data = json.load(f)

        # print((geojson_data))

        def read_geojson_data():
            with open(file_path, 'r') as f:
                geojson_data = json.load(f)
            return geojson_data

        def coordinates_calc():
            bbbox = BBOX
            return bbbox
        # Use the GeoJSON data as needed



        # get_ipython().system('sentinelhub.config --show')
        # get_ipython().run_line_magic('reload_ext', 'autoreload')
        # get_ipython().run_line_magic('autoreload', '2')
        # get_ipython().run_line_magic('matplotlib', 'inline')


        def plot_image(image, factor=1):
            """
            Utility function for plotting RGB images.
            """
            fig = plt.subplots(nrows=1, ncols=1, figsize=(15, 7))

            if np.issubdtype(image.dtype, np.floating):
                plt.imshow(np.minimum(image * factor, 1))
            else:
                plt.imshow(image)


        def dimensions():
            extent = coordinates_calc()
            print(extent)
            bbox = BBox(bbox=extent, crs=CRS.UTM_44N)
            centroid_x = (extent[0] + extent[2]) / 2
            centroid_y = (extent[1] + extent[3]) / 2
            centroid = [centroid_x, centroid_y]
            FAPAR = fapar_layer_call(bbox)
            transform = rasterio.transform.from_bounds(extent[0], extent[1], extent[2], extent[3], FAPAR.shape[1], FAPAR.shape[0])
            return bbox, centroid, transform


        #list (CustomUrlParam)


        #SENTINELHUB API WCS REQUEST FOR FAPAR LAYER
        def fapar_layer_call(bbox):
            wcs_true_color_request = WcsRequest(
                data_collection = DataCollection.SENTINEL2_L2A,
                data_folder = 'data/',
                layer = 'FAPAR_1',
                bbox = bbox,
                time = formatted_time,
                resx = '10m',
                resy = '10m',
                image_format = MimeType.TIFF,
                custom_url_params = {
                    CustomUrlParam.SHOWLOGO: False
                },
                config = config
            )
            wcs_true_color_img = wcs_true_color_request.get_data()
            FAPAR = wcs_true_color_img[0]
            return FAPAR


        #SENTINELHUB API WCS REQUEST FOR LSWI LAYER
        def LSWI_layer_call(bbox):
            wcs_true_color_request_LSWI = WcsRequest(
                data_collection = DataCollection.SENTINEL2_L2A,
                data_folder = 'data/',
                layer = 'LSWI',
                bbox = bbox,
                time =formatted_time,
                resx = '10m',
                resy = '10m',
                image_format = MimeType.TIFF,
                custom_url_params = {
                    CustomUrlParam.SHOWLOGO: False
                },
                config = config
            )
            wcs_true_color_img = wcs_true_color_request_LSWI.get_data()
            LSWI = wcs_true_color_img[0]
            return LSWI


        # w calculations
        def w_calc():
            bbox, centroid, transform = dimensions()
            LSWI = LSWI_layer_call(bbox)
            LSWI_max = np.amax(LSWI)
            w = (1 - LSWI) / (1 + LSWI_max)
            return w, transform


        # w plotting and saving tiff file
        w, transform = w_calc()
        # print(np.amax(w))
        # print(np.amin(w))
        # plt.imshow(w, cmap = 'RdYlGn')
        # plt.show()
        # with rasterio.open('/Users/sid/Desktop/Sugarcane_Sentinel-2_Products/Crop_Yield/W.tiff', 'w', driver = 'GTiff', width = w.shape[1], height = w.shape[0], count = 1, dtype = w.dtype, crs = 'EPSG:32644', transform = transform) as dst:
        #     dst.write(w, 1)


        # LUE CALCULATIONS
        def lue_calc():
            e0 = 3.22
            w, transform = w_calc()
            LUE = e0 * w
            return LUE, transform


        # LUE PLOTTING AND SAVING TIFF FILE
        LUE, transform = lue_calc()
        # print(np.amax(LUE))
        # print(np.amin(LUE))
        # plt.imshow(LUE, cmap = 'RdYlGn')
        # plt.show()
        # with rasterio.open('/Users/sid/Desktop/Sugarcane_Sentinel-2_Products/Crop_Yield/LUE.tiff', 'w', driver = 'GTiff', width = LUE.shape[1], height = LUE.shape[0], count = 1, dtype = LUE.dtype, crs = 'EPSG:32644', transform = transform) as dst:
        #     dst.write(LUE, 1)


        # UTM TO LAT-LONG CONVERSION
        def utm_to_latlong(easting, northing, zone_number, zone_letter):
            utm_proj = pyproj.Proj(proj="utm", zone=zone_number, ellps="WGS84", south=("N" > zone_letter))
            latlong_proj = pyproj.Proj(proj="latlong", ellps="WGS84")
            lon, lat = pyproj.transform(utm_proj, latlong_proj, easting, northing)
            return lat, lon


        #GLOBAL HORIZONTAL IRRADIANCE CALCULATIONS
        def ghi_calc():
            bbox, centroid, transform = dimensions()
            date = formatted_time
            latitude, longitude = utm_to_latlong(centroid[0], centroid[1], 44, "R")
            lat = latitude
            lon = longitude
            API_KEY = 'eb8cf696ffe68ba33f4b7c3b25e45d5c' #OpenWeatherMap @ARMS4AI
            #-------Call this API Once-------
            response = requests.get(f"https://api.openweathermap.org/energy/1.0/solar/data?lat={lat}&lon={lon}&date={date}&appid={API_KEY}")
            data = response.json()

            with open('data.json', 'w') as f:
                json.dump(data, f)

            f = open('data.json')
            j= json.load(f)
            GHI=0
            for i in j["irradiance"]["daily"]:
                GHI = i['clear_sky']['ghi']
            GHI = GHI/1000
            GHI = GHI * 3.6
            print(GHI)
            return GHI


        # NPP CALCULATIONS
        def npp_calc():
            bbox, centroid, transform = dimensions()
            FAPAR = fapar_layer_call(bbox)
            GHI = ghi_calc()
            LUE, transform = lue_calc()
            NPP = FAPAR * GHI * 0.5 * LUE
            nodata = 0
            NPP[NPP < 0] = nodata
            return NPP


        # NPP PLOTTING AND SAVING TIFF FILE
        NPP = npp_calc()
        # plt.imshow(NPP, cmap = 'RdYlGn')
        # plt.show()
        # with rasterio.open('/Users/sid/Desktop/Sugarcane_Sentinel-2_Products/Crop_Yield/NPP.tiff', 'w', driver = 'GTiff', width = NPP.shape[1], height = NPP.shape[0], count = 1, dtype = NPP.dtype, crs = 'EPSG:32644', transform = transform) as dst:
        #     dst.write(NPP, 1)


        # CROP YIELD CALCIULATIONS
        def cy_calc():
            NPP = npp_calc()
            cy = NPP * 0.8 *10
            return cy



        # CROP YIELD PLOTTING AND SAVING TIFF FILE
        cy = cy_calc()
        # plt.imshow(cy, cmap = 'RdYlGn')
        # plt.show()
        with rasterio.open('data/cy.tiff', 'w', driver = 'GTiff', width = cy.shape[1], height = cy.shape[0], count = 1, dtype = cy.dtype, crs = 'EPSG:32644', transform = transform) as dst:
            dst.write(cy, 1)
        # with rasterio.open(r'C:\Users\ANUBHAV\OneDrive\Desktop\Web App\reactAppArms4AI\reactapp\public\cyx.tiff', 'w', driver = 'GTiff', width = cy.shape[1], height = cy.shape[0], count = 1, dtype = cy.dtype, crs = 'EPSG:32644', transform = transform) as dst:
        #     dst.write(cy, 1)

        #CLIPPING RASTER 
        shp = gpd.read_file("data/output.shp")         ##add path to shapefile
        shp = shp.to_crs(epsg = 32644)
        with rasterio.open("data/cy.tiff") as src:    ##add path to CY.tiff file
            out_image, out_transform = rasterio.mask.mask(src, shp.geometry,crop=True)
            out_meta = src.meta.copy()
            out_meta.update({"driver": "GTiff",
                        "height": out_image.shape[1],
                        "width": out_image.shape[2],
                        "transform": out_transform})
            
        with rasterio.open("data/cy.tiff", "w", **out_meta) as dest:
            dest.write(out_image)

        # ZONAL STATISTICS
        def zonal_stats_calc():
            cy = zonal_stats(shp,
                    "data/cy.tiff",
                    band=1,
                                nodata=np.nan,
                                stats=['mean'], geojson_out=True)
            return cy[0]['properties']['mean']

        cy_mean = zonal_stats_calc() #send
        print(cy_mean)
        cy_mean = round(cy_mean,2)

        with open('cy_mean.json', 'w') as outfile:
            json.dump(cy_mean, outfile)
        return cy_mean

response = cropYeild(date, bbox)

tool_outputs = []
tool_outputs.append({"tool_call_id":tool_calls[0].id, "output": response})

In [41]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=tool_outputs
)

In [42]:
import time
while run.status not in ["completed", "failed","requires_action"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

In [43]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
for msg in messages:
    print(msg.content[0].text.value)

The number of homeless people in ABC in 2022 is 123.
How many people were homeless in ABC in 2022?
